PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.
    


In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql

In [2]:
con = sql.connect('sqlite_db_pythonsqlite.db')

In [3]:
#Q1: Some of the facilities charge a fee to members, but some do not.
#Write a SQL query to produce a list of the names of the facilities that do.


query = '''
SELECT name
FROM Facilities 
WHERE membercost != 0;
'''
df1 = pd.read_sql_query(query, con)
df1

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


In [4]:
#Q2: How many facilities do not charge a fee to members? */

query = '''
SELECT COUNT(name) AS FacWithFreeMembership FROM Facilities
WHERE membercost = 0;
'''

df2 = pd.read_sql_query(query, con)
df2

,FacWithFreeMembership
0,4


In [5]:
#Q3: Write an SQL query to show a list of facilities that charge a fee to members,
#where the fee is less than 20% of the facility's monthly maintenance cost.
#Return the facid, facility name, member cost, and monthly maintenance of the
#facilities in question. 

query = '''
SELECT facid, name AS facility_name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost < 0.8 * monthlymaintenance AND
membercost != 0
'''

df3 = pd.read_sql_query(query, con)
df3

,facid,facility_name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


In [6]:
#Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
#Try writing the query without using the OR operator.

query = '''
SELECT *
FROM Facilities
WHERE facid in (1,5)
'''

df4 = pd.read_sql_query(query, con)
df4

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


In [7]:
#Q5: Produce a list of facilities, with each labelled as
#'cheap' or 'expensive', depending on if their monthly maintenance cost is
#more than $100. Return the name and monthly maintenance of the facilities
#in question.

query = '''

SELECT name as facility, monthlymaintenance,
CASE
    WHEN monthlymaintenance > 100 THEN 'expensive'
    else 'cheap'
    END AS label
FROM Facilities
'''

df5 = pd.read_sql_query(query, con)
df5

,facility,monthlymaintenance,label
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

In [8]:
query = '''

SELECT firstname, surname AS lastname
FROM Members
WHERE joindate in ((SELECT MAX(joindate) from Members),(SELECT min(joindate) from Members))
'''

df6 = pd.read_sql_query(query, con)
df6

,firstname,lastname
0,GUEST,GUEST
1,Darren,Smith


Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [9]:
query = '''

SELECT DISTINCT firstname ||' ' ||surname AS guest, f.name
FROM Members as m
inner join Bookings as b
ON m.memid = b.memid
inner join Facilities as f
ON b.facid = f.facid
WHERE f.facid in (0,1)
ORDER BY guest
'''

df7 = pd.read_sql_query(query, con)
df7

,guest,name
0,Anne Baker,Tennis Court 1
1,Anne Baker,Tennis Court 2
2,Burton Tracy,Tennis Court 2
3,Burton Tracy,Tennis Court 1
4,Charles Owen,Tennis Court 1
5,Charles Owen,Tennis Court 2
6,Darren Smith,Tennis Court 2
7,David Farrell,Tennis Court 1
8,David Farrell,Tennis Court 2
9,David Jones,Tennis Court 2


/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

In [10]:
query = '''

SELECT f.name as facility, m.firstname ||' ' ||m.surname AS name, 
CASE WHEN m.memid = 0 THEN f.guestcost * b.slots 
ELSE f.membercost * b.slots END AS booking_cost
FROM Members as m
inner join Bookings as b
ON m.memid = b.memid
inner join Facilities as f
ON b.facid = f.facid
WHERE b.starttime like '%2012-09-14%' AND booking_cost > 30
ORDER BY booking_cost DESC
'''

df8 = pd.read_sql_query(query, con)
df8

,facility,name,booking_cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


Q9: This time, produce the same result as in Q8, but using a subquery.

In [11]:
query = '''

WITH book_sub AS (
SELECT f.name as facility, m.firstname ||' ' ||m.surname AS name, 
CASE WHEN m.memid = 0 THEN f.guestcost * b.slots 
ELSE f.membercost * b.slots END AS booking_cost
FROM Members as m
inner join Bookings as b
ON m.memid = b.memid
inner join Facilities as f
ON b.facid = f.facid
WHERE b.starttime like '%2012-09-14%'
) 
SELECT * FROM book_sub
WHERE booking_cost > 30
order by booking_cost DESC


'''

df8 = pd.read_sql_query(query, con)
df8

,facility,name,booking_cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Massage Room 1,GUEST GUEST,160.0
3,Massage Room 1,GUEST GUEST,160.0
4,Tennis Court 2,GUEST GUEST,150.0
5,Tennis Court 1,GUEST GUEST,75.0
6,Tennis Court 1,GUEST GUEST,75.0
7,Tennis Court 2,GUEST GUEST,75.0
8,Squash Court,GUEST GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [12]:
#facilities < 1000 revenue
#facility name , total revenue
#sorted by revenue

query = '''

WITH book_sub AS (
SELECT  *,
CASE WHEN m.memid = 0 THEN f.guestcost * b.slots 
ELSE f.membercost * b.slots END AS booking_cost
FROM Members as m
inner join Bookings as b
ON m.memid = b.memid
inner join Facilities as f
ON b.facid = f.facid
) 
SELECT name, SUM(booking_cost) as revenue
FROM book_sub
GROUP BY name
HAVING revenue < 1000
order by revenue DESC;



'''

df10 = pd.read_sql_query(query, con)
df10

,name,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order


In [13]:

query = '''
SELECT m1.surname ||', ' ||m1.firstname AS name, m2.surname ||', ' ||m2.firstname AS recommended_by
FROM Members as m1
LEFT JOIN Members as m2
WHERE m1.recommendedby = m2.memid
ORDER BY name;
'''

df11 = pd.read_sql_query(query, con)
df11

,name,recommended_by
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


Q12: Find the facilities with their usage by member, but not guests


In [14]:
query = '''
SELECT b.facid, COUNT( b.memid ) AS member_usage, f.name
FROM (
SELECT facid, memid
FROM Bookings
WHERE memid !=0
) AS b
LEFT JOIN Facilities AS f ON b.facid = f.facid
GROUP BY b.facid;
'''
df12 = pd.read_sql_query(query, con)
df12

,facid,member_usage,name
0,0,308,Tennis Court 1
1,1,276,Tennis Court 2
2,2,344,Badminton Court
3,3,385,Table Tennis
4,4,421,Massage Room 1
5,5,27,Massage Room 2
6,6,195,Squash Court
7,7,421,Snooker Table
8,8,783,Pool Table


Q13: Find the facilities usage by month, but not guests



In [18]:
query = '''
SELECT strftime('%m',b.starttime) AS month, f.name,COUNT( b.memid ) AS member_usage
FROM Bookings as b
LEFT JOIN Facilities as f
ON b.facid = f.facid
WHERE memid != 0
GROUP BY month, f.name
'''
df13 = pd.read_sql_query(query, con)
df13

,month,name,member_usage
0,07,Badminton Court,51
1,07,Massage Room 1,77
2,07,Massage Room 2,4
3,07,Pool Table,103
4,07,Snooker Table,68
5,07,Squash Court,23
6,07,Table Tennis,48
7,07,Tennis Court 1,65
8,07,Tennis Court 2,41
9,08,Badminton Court,132
